In [ ]:
# Replace 'your_file.zip' with the name of your zip file
!unzip shirts.zip -d /content/

In [ ]:
pip install ultralytics

In [ ]:
!pip install albumentations opencv-python

In [ ]:
import os
import cv2
import albumentations as A
from glob import glob

# Create directories for augmented data
augmented_images_path = '/content/augmented1/images/train'
augmented_labels_path = '/content/augmented1/labels/train'

os.makedirs(augmented_images_path, exist_ok=True)
os.makedirs(augmented_labels_path, exist_ok=True)

# Define augmentations
transform = A.Compose([
    A.OneOf([
        A.HorizontalFlip(p=0.5),  # Flip image horizontally
        A.VerticalFlip(p=0.5),    # Flip image vertically
    ], p=0.7),  # Apply one of the flips with 50% probability

    A.RandomBrightnessContrast(
        brightness_limit=0.6,  # Â±40%
        contrast_limit=0.7,    # Â±40%
        p=1.0),  # Apply brightness and contrast adjustments with 100% probability

    A.HueSaturationValue(
        hue_shift_limit=0.035,  # Adjust hue
        sat_shift_limit=0.8,    # Adjust saturation
        val_shift_limit=0.6,    # Adjust brightness
        p=1.0),  # Apply HSV adjustments with 100% probability

    A.Perspective(
        scale=(0.07, 0.4),  # Apply perspective transformations
        p=0.7),  # Apply perspective transformations with 50% probability
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))  # Specify how to handle bboxes

# Paths to your images and labels
images_path = '/content/shirts/images/train'
labels_path = '/content/shirts/labels/train'

# Define the image extensions you want to include
image_extensions = ['.jpg', '.jpeg', '.png', '.webp']

# Collect all image files with the specified extensions
images = []
for ext in image_extensions:
    images.extend(glob(os.path.join(images_path, f'*{ext}')))

for image_path in images:
    # Load image
    image = cv2.imread(image_path)
    image_name = os.path.basename(image_path)
    label_name = os.path.splitext(image_name)[0] + '.txt'

    # Load corresponding label
    label_path = os.path.join(labels_path, label_name)
    if not os.path.exists(label_path):
        continue

    # Read label file
    with open(label_path, 'r') as f:
        bboxes = []
        class_labels = []  # To store class labels for each bbox
        for line in f:
            class_id, x_center, y_center, width, height = map(float, line.strip().split())
            bboxes.append([x_center, y_center, width, height])
            class_labels.append(int(class_id))

    # Save the original image and label
    original_image_path = os.path.join(augmented_images_path, image_name)
    cv2.imwrite(original_image_path, image)

    original_label_path = os.path.join(augmented_labels_path, label_name)
    with open(original_label_path, 'w') as f:
        for bbox, class_id in zip(bboxes, class_labels):
            x_center, y_center, width, height = bbox
            f.write(f'{class_id} {x_center} {y_center} {width} {height}\n')

    for i in range(3):  # Apply augmentations 3 times
        # Apply augmentations
        transformed = transform(image=image, bboxes=bboxes, class_labels=class_labels)  # Pass class labels
        transformed_image = transformed['image']
        transformed_bboxes = transformed['bboxes']
        transformed_class_labels = transformed['class_labels']  # Get transformed class labels

        # Save augmented image
        augmented_image_path = os.path.join(augmented_images_path, f"{os.path.splitext(image_name)[0]}_aug_{i}.jpg")
        cv2.imwrite(augmented_image_path, transformed_image)

        # Save augmented labels
        augmented_label_path = os.path.join(augmented_labels_path, f"{os.path.splitext(label_name)[0]}_aug_{i}.txt")
        with open(augmented_label_path, 'w') as f:
            for bbox, class_id in zip(transformed_bboxes, transformed_class_labels):
                x_center, y_center, width, height = bbox
                f.write(f'{class_id} {x_center} {y_center} {width} {height}\n')

print("Augmentation completed.")


Augmentation completed.


In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from glob import glob
from random import sample

def plot_image_with_bboxes(ax, image, bboxes):
    ax.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    for bbox in bboxes:
        x_center, y_center, width, height, class_id = bbox
        # Convert from relative to absolute coordinates
        x_min = int((x_center - width / 2) * image.shape[1])
        y_min = int((y_center - height / 2) * image.shape[0])
        rect_width = int(width * image.shape[1])
        rect_height = int(height * image.shape[0])

        rect = patches.Rectangle((x_min, y_min), rect_width, rect_height, linewidth=1, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
    ax.axis('off')  # Hide the axes

# Paths to your augmented images and labels
images_path = '/content/augmented1/images/train'
labels_path = '/content/augmented1/labels/train'

# Collect all image files
image_files = glob(os.path.join(images_path, '*.*'))  # Wildcard for all extensions

# Randomly select 15 images
sample_images = sample(image_files, 15)

# Set up the subplot grid
fig, axes = plt.subplots(3, 5, figsize=(20, 12))  # Adjust figsize as needed
axes = axes.flatten()

for i, image_path in enumerate(sample_images):
    # Load image
    image = cv2.imread(image_path)

    # Check if the image was loaded properly
    if image is None:
        print(f"Error loading image: {image_path}")
        continue

    # Load corresponding label
    label_name = os.path.splitext(os.path.basename(image_path))[0] + '.txt'
    label_path = os.path.join(labels_path, label_name)

    bboxes = []
    if os.path.exists(label_path):
        with open(label_path, 'r') as f:
            for line in f:
                class_id, x_center, y_center, width, height = map(float, line.strip().split())
                bboxes.append([x_center, y_center, width, height, int(class_id)])

    # Plot image with bounding boxes
    plot_image_with_bboxes(axes[i], image, bboxes)

# Hide any unused subplots
for j in range(len(sample_images), len(axes)):
    axes[j].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
import os

def count_images_in_folder(folder_path, image_extensions):
    # Initialize count
    image_count = 0

    # Iterate through all files in the folder
    for file_name in os.listdir(folder_path):
        # Check if the file has an image extension
        if any(file_name.lower().endswith(ext) for ext in image_extensions):
            image_count += 1

    return image_count

# Define the path to your folder and the image extensions to look for
folder_path = '/content/augmented1/images/train'
image_extensions = ['.jpg', '.jpeg', '.png', '.webp']

# Get the count of images
num_images = count_images_in_folder(folder_path, image_extensions)

print(f"Number of images in the folder: {num_images}")

Number of images in the folder: 512


In [ ]:
ROOT_DIR = '/content/'

In [ ]:
import os

from ultralytics import YOLO

# Load a model
model = YOLO('yolov8n.yaml')

# Use the model
results = model.train(data=os.path.join(ROOT_DIR, "shirts.yaml"), epochs=20)

Ultralytics YOLOv8.2.69 ðŸš€ Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=/content/shirts.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train10, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show

train: Scanning /content/augmented1/labels/train... 512 images, 0 backgrounds, 0 corrupt: 100%|â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆ| 512/512 [00:00<00:00, 1591.83it/s]

train: New cache created: /content/augmented1/labels/train.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/shirts/labels/val.cache... 32 images, 0 backgrounds, 0 corrupt: 100%|â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆ| 32/32 [00:00<?, ?it/s]


Plotting labels to runs/detect/train10/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.00125, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added âœ…
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/train10
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      20/20      2.25G     0.9162      1.581      1.799         16        640: 100%|â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆ| 32/32 [00:11<00:00,  2.81it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆ| 1/1 [00:00<00:00,  2.62it/s]

                   all         32         32      0.788       0.75      0.845      0.649



20 epochs completed in 0.062 hours.
Optimizer stripped from runs/detect/train10/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train10/weights/best.pt, 6.2MB

Validating runs/detect/train10/weights/best.pt...
Ultralytics YOLOv8.2.69 ðŸš€ Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8n summary (fused): 168 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆ| 1/1 [00:00<00:00,  4.05it/s]


                   all         32         32      0.672      0.896       0.85       0.65
           Black_Shirt          8          8      0.603          1      0.954      0.771
            Blue_Shirt          8          8      0.891          1      0.995      0.786
           White_Shirt          8          8      0.885      0.963      0.967      0.712
            Grey_Shirt          8          8      0.312      0.622      0.484      0.333
Speed: 0.2ms preprocess, 1.9ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs/detect/train10
